In [1]:
%matplotlib inline
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

#Standardization
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('Plane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Hourse', 'Ship', 'Truck')

# Show the img
def imshow(img):
    img = img / 2 + 0.5     
    # Anti-Standardization
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

Files already downloaded and verified


In [3]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        #pooling 16 
        self.conv3 = nn.Conv2d(64, 192, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(192)
        self.conv4 = nn.Conv2d(192, 192, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(192)
        #pooling 8 
        self.conv5 = nn.Conv2d(192, 384, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(384)
        self.conv6 = nn.Conv2d(384, 256, 3, padding=1)
        self.bn6 = nn.BatchNorm2d(256)
        self.conv7 = nn.Conv2d(256, 256, 3)
        self.bn7 = nn.BatchNorm2d(256)
        #pooling 3
        self.dropout = nn.Dropout()
        self.fc1 = nn.Linear(256 * 3 * 3, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 10)

    def forward(self, x):
        x = F.sigmoid(self.bn1(self.conv1(x)))*(self.bn1(self.conv1(x)))
        x = F.max_pool2d((F.sigmoid(self.bn2(self.conv2(x)))*(self.bn2(self.conv2(x)))), (2, 2))
        x = F.sigmoid(self.bn3(self.conv3(x)))*(self.bn3(self.conv3(x)))
        x = F.max_pool2d((F.sigmoid(self.bn4(self.conv4(x)))*(self.bn4(self.conv4(x)))), (2, 2))
        x = F.sigmoid(self.bn5(self.conv5(x)))*(self.bn5(self.conv5(x)))
        x = F.sigmoid(self.bn6(self.conv6(x)))*(self.bn6(self.conv6(x)))
        x = F.max_pool2d((F.sigmoid(self.bn7(self.conv7(x)))*(self.bn7(self.conv7(x)))), (2, 2))
        x = x.view(x.size(0), 256 * 3 * 3)
        x = self.dropout(x)
        x = F.sigmoid(self.fc1(x))*self.fc1(x)
        x = F.sigmoid(self.fc2(x))*self.fc2(x)
        x = self.fc3(x)
        return x

net = Net1()
print(net)

Net1 (
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (conv3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
  (conv4): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
  (conv5): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True)
  (conv6): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (conv7): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
  (bn7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (dropout): Dro

In [4]:
# Cross EntropyLoss
criterion = nn.CrossEntropyLoss()
# Optimizer：SGD
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [23]:
# An epoch means going through all data once 6 epoches 81% 
for epoch in range(1):  

    running_loss = 0.0
# enumerate is a function in python that enumerate batches
    for i, data in enumerate(trainloader, 0):
        # Output
        inputs, labels = data
        
        # Variable 
        inputs, labels = Variable(inputs), Variable(labels)

        # Set gradients of Variable as zero
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.261
[1,  4000] loss: 0.282
[1,  6000] loss: 0.306
[1,  8000] loss: 0.298
[1, 10000] loss: 0.302
[1, 12000] loss: 0.319
Finished Training


In [24]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    labels = labels
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 81 %


In [25]:
#36
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    labels = labels
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1


for i in range(10):
    print(' %5s Prediction Accuarcy: %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

 飞机 Prediction Accuarcy: 86 %
 汽车 Prediction Accuarcy: 91 %
   鸟 Prediction Accuarcy: 69 %
   猫 Prediction Accuarcy: 55 %
   鹿 Prediction Accuarcy: 80 %
   狗 Prediction Accuarcy: 84 %
 青蛙 Prediction Accuarcy: 88 %
   马 Prediction Accuarcy: 84 %
   船 Prediction Accuarcy: 84 %
 卡车 Prediction Accuarcy: 88 %


In [26]:
#dropout Switch
net.dropout = nn.Dropout(p=0.5)
net

Net1 (
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (conv3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
  (conv4): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
  (conv5): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True)
  (conv6): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (conv7): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
  (bn7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (dropout): Dro

In [27]:
#dropout Switch
net.dropout = nn.Dropout(p=0)
correct = 0
total = 0
for data in testloader:
    images, labels = data
    labels = labels
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 81 %
